In [1]:
import numpy as np
from pomegranate import *
import itertools

### Trying bayes net in pomegranate

In [2]:
#generate example data
n = 10
h = BernoulliDistribution(0.3).sample(n)
w = BernoulliDistribution(0.6).sample(n)
l = BernoulliDistribution(0.7).sample(n) 

In [3]:
# 1. Initialize parameters (uniform)

height = DiscreteDistribution({'short':0.5, 'tall':0.5})
weight = DiscreteDistribution({'low':0.5, 'high':0.5})
sex = ConditionalProbabilityTable([
    ['short', 'low', 'm', 0.5],
    ['short', 'low', 'f', 0.5],
    ['short', 'high', 'm', 0.5],
    ['short', 'high', 'f', 0.5],
    ['tall', 'low', 'm', 0.5],
    ['tall', 'low', 'f', 0.5],
    ['tall', 'high', 'm', 0.5],
    ['tall', 'high', 'f', 0.5]], [height, weight] )
hair = ConditionalProbabilityTable([
    ['m', 'sh', 0.5],
    ['m', 'long', 0.5],
    ['f', 'sh', 0.5],
    ['f', 'long', 0.5]], [sex] )
    

In [4]:
# create state objects (nodes)

s_h = State(height, name='height')
s_w = State(weight, name='weight')
s_s = State(sex, name='sex')
s_l = State(hair, name='hair')

In [5]:
# create the model

model = BayesianNetwork("Hair Length")

# Add the three states to the network 

model.add_states(s_h, s_w, s_s, s_l)

In [6]:
# build the DAG

model.add_transition(s_h, s_s)
model.add_transition(s_w, s_s)
model.add_transition(s_s, s_l)
model.bake()

In [13]:
model.predict_proba({'height':'short','weight':'low'})

array([ {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "tall" :0.0,
            "short" :1.0
        }
    ],
    "name" :"DiscreteDistribution"
},
       {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "high" :0.0,
            "low" :1.0
        }
    ],
    "name" :"DiscreteDistribution"
},
       {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "m" :0.5,
            "f" :0.5
        }
    ],
    "name" :"DiscreteDistribution"
},
       {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "sh" :0.5,
            "long" :0.5
        }
    ],
    "name" :"DiscreteDistribution"
}], dtype=object)

In [8]:
[state.name for state in model.states]

['height', 'weight', 'sex', 'hair']

In [19]:
s_s.distribution.column_idxs

array([0, 1, 2], dtype=int32)

In [20]:
s_l.distribution.column_idxs

array([2, 3], dtype=int32)

### A simple model for trying things 

In [24]:
A = DiscreteDistribution({1:0.45,0:0.55})
B = ConditionalProbabilityTable([
    [1, 0, 0.3],
    [1, 1, 0.7],
    [0, 0, 0.4],
    [0, 1, 0.6]], [A] )

In [25]:
s_A = State(A, name='A')
s_B = State(B, name='B')

In [26]:
test_model = BayesianNetwork("test")
test_model.add_states(s_A, s_B)
test_model.add_transition(s_A, s_B)
test_model.bake()

In [5]:
name_list = [s.name for s in test_model.states]

In [36]:
test_model.states[0].distribution.parameters[0][0]=0.99

In [37]:
test_model.states[0].distribution.parameters

[{0: 0.99, 1: 0.45}]

In [38]:
test_model.states

[{
     "distribution" : {
         "frozen" : false,
         "class" : "Distribution",
         "parameters" : [
             {
                 "1" : 0.45,
                 "0" : 0.99
             }
         ],
         "name" : "DiscreteDistribution"
     },
     "class" : "State",
     "weight" : 1.0,
     "name" : "A"
 }, {
     "distribution" : {
         "table" : [
             [
                 "0",
                 "0",
                 "0.4"
             ],
             [
                 "0",
                 "1",
                 "0.6"
             ],
             [
                 "1",
                 "0",
                 "0.3"
             ],
             [
                 "1",
                 "1",
                 "0.7"
             ]
         ],
         "parents" : [
             {
                 "frozen" : false,
                 "class" : "Distribution",
                 "parameters" : [
                     {
                         "1" : 0.45,
          

In [6]:
# getting the conditional probability

prob_B = test_model.predict_proba({'A':1})[1]

In [7]:
prob_B

{
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "1" :0.6999999999999998,
            "0" :0.3000000000000001
        }
    ],
    "name" :"DiscreteDistribution"
}

In [8]:
prob_B.probability(1)

0.69999999999999984

In [20]:
data = np.asarray([[0,1],[0,0],[1,1],[1,1],[0,0],[1,0],[0,0]])

In [19]:
s_A.distribution.marginal()

{
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "1" :0.42857142857142855,
            "0" :0.5714285714285714
        }
    ],
    "name" :"DiscreteDistribution"
}

In [22]:
test_model.fit(data)

{
    "states" : [
        {
            "distribution" : {
                "frozen" : false,
                "class" : "Distribution",
                "parameters" : [
                    {
                        "1" : 0.42857142857142855,
                        "0" : 0.5714285714285714
                    }
                ],
                "name" : "DiscreteDistribution"
            },
            "class" : "State",
            "weight" : 1.0,
            "name" : "A"
        },
        {
            "distribution" : {
                "table" : [
                    [
                        "1",
                        "0",
                        "0.333333333333"
                    ],
                    [
                        "1",
                        "1",
                        "0.666666666667"
                    ],
                    [
                        "0",
                        "0",
                        "0.75"
                    ],
                   

In [11]:
test_model.predict_proba()

array([ {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "1" :0.42857142857142866,
            "0" :0.5714285714285714
        }
    ],
    "name" :"DiscreteDistribution"
},
       {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "1" :0.4285714285714286,
            "0" :0.5714285714285713
        }
    ],
    "name" :"DiscreteDistribution"
}], dtype=object)

In [15]:
type(test_model)

pomegranate.BayesianNetwork.BayesianNetwork

### Model for testing 2

In [12]:
A = DiscreteDistribution({'1':0.2, '0':0.8})
H = ConditionalProbabilityTable([
        ['0', '0', 0.4],
        ['0', '1', 0.6],
        ['1', '0', 0.3],
        ['1', '1', 0.7]], [A])
B = ConditionalProbabilityTable([
        ['0', '0', 0.6],
        ['0', '1', 0.4],
        ['1', '0', 0.5],
        ['1', '1', 0.5]], [H])

In [13]:
s_A = State(A, name='A')
s_H = State(H, name='H')
s_B = State(B, name='B')

In [14]:
test_model2 = BayesianNetwork("test2")
test_model2.add_states(s_A, s_H, s_B)
test_model2.add_transition(s_A, s_H)
test_model2.add_transition(s_H, s_B)
test_model2.bake()

Data:

In [15]:
data2 = np.array([[0, np.nan, 0],
                   [0, np.nan, 1],
                   [1, np.nan, 0],
                   [1, np.nan, 1],
                   [1, np.nan, 0],
                   [0, np.nan, 1],
                   [1, np.nan, 1],
                   [1, np.nan, 0]], dtype=str)

In [6]:
data2[:, [0,2]]

array([['0', '0'],
       ['0', '1'],
       ['1', '0'],
       ['1', '1'],
       ['1', '0'],
       ['0', '1'],
       ['1', '1'],
       ['1', '0']], 
      dtype='|S3')

Test:

In [7]:
mb2 = MarkovBlanket(1)
mb2.populate(test_model2)
mb2.calculate_prob(test_model2)

NameError: name 'MarkovBlanket' is not defined

In [86]:
mb2.prob_table

{1: {(0, 0): 0.4, (0, 1): 0.6, (1, 0): 0.3, (1, 1): 0.7},
 2: {(0, 0): 0.6, (0, 1): 0.4, (1, 0): 0.5, (1, 1): 0.5}}

In [87]:
expected_counts2 = create_expected_counts(test_model2, mb2) 

In [88]:
expected_counts2

{(0, 0, 0): 0,
 (0, 0, 1): 0,
 (0, 1, 0): 0,
 (0, 1, 1): 0,
 (1, 0, 0): 0,
 (1, 0, 1): 0,
 (1, 1, 0): 0,
 (1, 1, 1): 0}

In [110]:
expected_counts2 = update_expected_counts(expected_counts2, test_model2, mb2)

{(0, 1): 0.46, (1, 0): 0.53, (0, 0): 0.54, (1, 1): 0.47}


In [111]:
expected_counts2

{(0, 0, 0): 0.4444444444444444,
 (0, 0, 1): 0.3478260869565218,
 (0, 1, 0): 0.33962264150943394,
 (0, 1, 1): 0.2553191489361702,
 (1, 0, 0): 0.5555555555555555,
 (1, 0, 1): 0.6521739130434782,
 (1, 1, 0): 0.660377358490566,
 (1, 1, 1): 0.7446808510638298}

In [105]:
mb2.parents

[0]

## Building the EM

Steps:
1. Initialize parameters with near uniform values.
2. Build the model with the initialized parameters. 
3. Expectation: Calculate the expected counts for the hidden variables. Update the data for the hidden variables. 
4. Maximization: Calculate MLE for all parameters by summing over expected counts. 
5. Update the model with the new parameter values.
6. Iterate 3-5 until converges.


In [16]:
def search_hidden(data):
    """Returns the column index of the hidden node if only one column of NaN.
    
    Parameters
    ----------
    data : An ndarray (n_sample, n_nodes)
    
    Returns
    -------
    ind_h : the index of the hidden node column
    """
    is_col_nan = np.all(np.isnan(data), axis=0)
    ind = np.where(is_col_nan)
    if np.size(ind)==1:
        ind_h = ind[0][0]
    else:
        raise ValueError('Data contains more than one hidden nodes or no hidden node')
    return ind_h
    

In [17]:
class MarkovBlanket():
    
    def __init__(self, ind_h):
        self.hidden = ind_h
        self.parents = []
        self.children = []
        self.coparents = []
        self.prob_table = {}
        
    def populate(self, model):
        state_indices = {state.name : i for i, state in enumerate(model.states)}
        
        edges_list = [(parent.name, child.name) for parent, child in model.edges]
        edges_list = [(state_indices[parent],state_indices[child]) 
                  for parent, child in edges_list]
        
        self.children = list(set([child for parent, child in edges_list if parent==self.hidden]))
        self.parents = list(set([parent for parent, child in edges_list if child==self.hidden]))
        self.coparents = list(set([parent for parent, child in edges_list if child in self.children]))
        try:
            self.coparents.remove(self.hidden)
        except ValueError:
            pass
            
    def calculate_prob(self, model):
        """Create the probability table from states
        """
        for ind_state in [self.hidden]+self.children:
            distribution = model.states[ind_state].distribution
            
            if isinstance(distribution, ConditionalProbabilityTable):
                table = distribution.parameters[0]
                self.prob_table[ind_state] = {tuple(row[:-1]) : row[-1] for row in table}
            else:
                self.prob_table[ind_state] = distribution.parameters[0]
                
    def update_prob(self, model, expected_counts, ct):
        """Update the probability table using expected counts
        """
        ind = {x : i for i, x in enumerate([mb.hidden] + mb.parents + mb.children + mb.coparents)}
        
        for ind_state in [mb.hidden] + mb.children:
            distribution = model.states[ind_state].distribution
            
            if isinstance(distribution, ConditionalProbabilityTable):
                parents = distribution.column_idxs
                table = self.prob_table[ind_state] # dict
                
                for key in table.keys():
                    # calculate the new parameter for this key
                    
        
        

In [18]:
class ExpectedCounts():
    
    def __init__(self, model, mb):
        self.counts = {}
        
        self.populate(model, mb)
        
    def populate(self, model, mb):
        #create combinations of keys
        keys_list = [model.states[mb.hidden].distribution.keys()]
        for ind in mb.parents + mb.children + mb.coparents:
            keys_list.append(model.states[ind].distribution.keys())
        
        self.counts = {p:0 for p in itertools.product(*keys_list)}
        
    def update(self, model, mb):
        ind = {x : i for i, x in enumerate([mb.hidden] + mb.parents + mb.children + mb.coparents)}
    
        marginal_prob = {}
    
        # calculate joint probability and marginal probability
        for i, key in enumerate(self.counts.keys()):
            prob = 1
        
            for j, ind_state in enumerate([mb.hidden] + mb.children):
                distribution = model.states[ind_state].distribution
            
                if isinstance(distribution, ConditionalProbabilityTable):
                    idxs = distribution.column_idxs
                    state_key = tuple([key[ind[x]] for x in idxs])
                else:
                    state_key = key[ind[ind_state]]
                
                prob = prob*mb.prob_table[ind_state][state_key]         
                self.counts[key] = prob
            try:
                marginal_prob[key[1:]] += prob
            except KeyError:
                marginal_prob[key[1:]] = prob
                 
        # divide the joint prob by the marginal prob to get the conditional
        for i, key in enumerate(self.counts.keys()):
            self.counts[key] = self.counts[key]/marginal_prob[key[1:]]
    
        

In [19]:
class CountTable():
    
    def __init__(self, model, mb, items):
        self.table ={}
        
        self.populate(model, mb, items)
        
    def populate(self, model, mb, items):
        keys_list = []
        for ind in mb.parents + mb.children + mb.coparents:
            keys_list.append(model.states[ind].distribution.keys())
        
        # init
        self.table = {p:0 for p in itertools.product(*keys_list)}
        
        # count
        for i, row in enumerate(items):
            try:
                self.table[tuple(row)] += 1
            except KeyError:
                print 'Items in row', i, 'does not match the set of keys.'
                raise KeyError
    

In [20]:
def update_nodes(model, mb, ct):
    """Update parameters for hidden and children nodes
    """
    for ind_state in [mb.hidden] + mb.children:
        distribution = model.states[ind_state]
        
        if isinstance(distribution, ConditionalProbabilityTable):
            idxs = distribution.column_idxs
            
    

In [29]:
test_model2.states[1].distribution.parameters[0][0][2] = 0.5

In [33]:
test_model2.states[2].distribution.parameters[0][0][2] = 0.55

In [60]:
test_model2.states[2].distribution.parameters

[[['0', '0', 0.55], ['0', '1', 0.4], ['1', '0', 0.5], ['1', '1', 0.5]], [{
      "table" : [
          [
              "1",
              "1",
              "0.7"
          ],
          [
              "1",
              "0",
              "0.3"
          ],
          [
              "0",
              "1",
              "0.6"
          ],
          [
              "0",
              "0",
              "0.4"
          ]
      ],
      "parents" : [
          {
              "frozen" : false,
              "class" : "Distribution",
              "parameters" : [
                  {
                      "1" : 0.2,
                      "0" : 0.8
                  }
              ],
              "name" : "DiscreteDistribution"
          }
      ],
      "class" : "Distribution",
      "name" : "ConditionalProbabilityTable"
  }]]

In [58]:
test_model2.bake()

In [59]:
test_model2.states[1].distribution.parameters

[[['0', '0', 0.5], ['0', '1', 0.6], ['1', '0', 0.3], ['1', '1', 0.7]], [{
      "frozen" :false,
      "class" :"Distribution",
      "parameters" :[
          {
              "1" :0.2,
              "0" :0.8
          }
      ],
      "name" :"DiscreteDistribution"
  }]]

In [17]:
mb = MarkovBlanket(1)
mb.populate(test_model2)
mb.calculate_prob(test_model2)

In [15]:
items = data2[:, [0,2]]

In [18]:
table = CountTable(test_model2, mb, items)

In [19]:
table.table

{('0', '0'): 1, ('0', '1'): 2, ('1', '0'): 3, ('1', '1'): 2}

In [25]:
table.table.keys()[0][0]

'1'

In [29]:
mb = MarkovBlanket(2)

In [30]:
mb.populate(model)

In [31]:
mb.calculate_prob(model)

In [53]:
mb.prob_table[3]

{('f', 'long'): 0.5, ('f', 'sh'): 0.5, ('m', 'long'): 0.5, ('m', 'sh'): 0.5}

In [50]:
expected_counts = create_expected_counts(model, mb)

In [52]:
expected_counts

{('f', 'short', 'high', 'long'): 0.25,
 ('f', 'short', 'high', 'sh'): 0.25,
 ('f', 'short', 'low', 'long'): 0.25,
 ('f', 'short', 'low', 'sh'): 0.25,
 ('f', 'tall', 'high', 'long'): 0.25,
 ('f', 'tall', 'high', 'sh'): 0.25,
 ('f', 'tall', 'low', 'long'): 0.25,
 ('f', 'tall', 'low', 'sh'): 0.25,
 ('m', 'short', 'high', 'long'): 0.25,
 ('m', 'short', 'high', 'sh'): 0.25,
 ('m', 'short', 'low', 'long'): 0.25,
 ('m', 'short', 'low', 'sh'): 0.25,
 ('m', 'tall', 'high', 'long'): 0.25,
 ('m', 'tall', 'high', 'sh'): 0.25,
 ('m', 'tall', 'low', 'long'): 0.25,
 ('m', 'tall', 'low', 'sh'): 0.25}

In [12]:
model.states[2].distribution.column_idxs

array([0, 1, 2], dtype=int32)

In [18]:
model.states[2].distribution.parameters[0]

[['short', 'low', 'm', 0.5],
 ['short', 'low', 'f', 0.5],
 ['short', 'high', 'm', 0.5],
 ['short', 'high', 'f', 0.5],
 ['tall', 'low', 'm', 0.5],
 ['tall', 'low', 'f', 0.5],
 ['tall', 'high', 'm', 0.5],
 ['tall', 'high', 'f', 0.5]]

In [14]:
expected_counts.keys()

[('m', 'tall', 'low', 'sh'),
 ('f', 'tall', 'high', 'long'),
 ('m', 'short', 'low', 'sh'),
 ('f', 'tall', 'low', 'long'),
 ('f', 'short', 'low', 'long'),
 ('f', 'tall', 'high', 'sh'),
 ('m', 'tall', 'low', 'long'),
 ('f', 'tall', 'low', 'sh'),
 ('m', 'short', 'low', 'long'),
 ('f', 'short', 'high', 'long'),
 ('m', 'tall', 'high', 'long'),
 ('f', 'short', 'high', 'sh'),
 ('m', 'short', 'high', 'long'),
 ('f', 'short', 'low', 'sh'),
 ('m', 'tall', 'high', 'sh'),
 ('m', 'short', 'high', 'sh')]

In [51]:
expected_counts

{('f', 'short', 'high', 'long'): 0,
 ('f', 'short', 'high', 'sh'): 0,
 ('f', 'short', 'low', 'long'): 0,
 ('f', 'short', 'low', 'sh'): 0,
 ('f', 'tall', 'high', 'long'): 0,
 ('f', 'tall', 'high', 'sh'): 0,
 ('f', 'tall', 'low', 'long'): 0,
 ('f', 'tall', 'low', 'sh'): 0,
 ('m', 'short', 'high', 'long'): 0,
 ('m', 'short', 'high', 'sh'): 0,
 ('m', 'short', 'low', 'long'): 0,
 ('m', 'short', 'low', 'sh'): 0,
 ('m', 'tall', 'high', 'long'): 0,
 ('m', 'tall', 'high', 'sh'): 0,
 ('m', 'tall', 'low', 'long'): 0,
 ('m', 'tall', 'low', 'sh'): 0}

In [12]:
state_indices = {state.name : i for i, state in enumerate(model.states)}
parents, children, coparents = markov_blanket(model,2,state_indices)

In [53]:
for state in model.states:
    print state.distribution.keys()

('tall', 'short')
('high', 'low')
('m', 'f')
('sh', 'long')


In [52]:
get_markov_blanket(model, 2, state_indices)

coparents: set([2])


[0, 1, 2, 3]

In [34]:
state_indices = {state.name : i for i, state in enumerate(model.states)}

In [29]:
for parent, child in model.edges:
    print parent.name,'-->',child.name

height --> sex
weight --> sex
sex --> hair


In [8]:
###DEPRICATED

def create_expected_counts(model, mb):
    """Returns a dict of expected counts of the hidden node init to zeros
    
    Parameter
    ---------
    model : BayesianNetwork object
    
    mb : MarkovBlanket object
        
    Returns
    -------
    counts : dict
    """
    #create combinations of keys
    keys_list = [model.states[mb.hidden].distribution.keys()]
    for ind in mb.parents + mb.children + mb.coparents:
        keys_list.append(model.states[ind].distribution.keys())
        
    counts = {p:0 for p in itertools.product(*keys_list)}
    
    return counts

def update_expected_counts(expected_counts, model, mb):
    """Returns a dict of updated expected counts of the hidden node
    
    Parameter
    ---------
    expected_counts : dict
    
    model : BayesianNetwork object
    
    mb : MarkovBlanket object
        
    Returns
    -------
    expected_counts : dict
    
    """         
    ind = {x : i for i, x in enumerate([mb.hidden] + mb.parents + mb.children + mb.coparents)}
    
    marginal_prob = {}
    
    # calculate joint probability and marginal probability
    for i, key in enumerate(expected_counts.keys()):
        prob = 1
        
        for j, ind_state in enumerate([mb.hidden] + mb.children):
            distribution = model.states[ind_state].distribution
            
            if isinstance(distribution, ConditionalProbabilityTable):
                idxs = distribution.column_idxs
                state_key = tuple([key[ind[x]] for x in idxs])
            else:
                state_key = key[ind[ind_state]]
                
            prob = prob*mb.prob_table[ind_state][state_key]         
            expected_counts[key] = prob
        try:
            marginal_prob[key[1:]] += prob
        except KeyError:
            marginal_prob[key[1:]] = prob
                 
    # divide the joint prob by the marginal prob to get the conditional
    for i, key in enumerate(expected_counts.keys()):
        expected_counts[key] = expected_counts[key]/marginal_prob[key[1:]]
            
    return expected_counts
    

In [70]:
# EM algorithm for hidden variables assuming Bernoulli Distribution (not supporting missing cells)

def em(model, data, max_iter):
    """Returns a model with added parameters for hidden nodes.
    
    Parameters
    ----------
    model : A BayesianNetwork object
    
    data : array-like, shape (n_sample, n_nodes)
        The data array where each row is a sample and each column represents a node.
        The order of the columns should be the same as the order for the nodes in 
        the model. Hidden nodes should have NaN values.
    
    max_iter : int
        The maximum number of iterations to run the EM algorithm for.
        
    Returns
    -------
    model : A BayesianNetwork object
    
    """
    
    # all possible values of data
    data_perm = [x for x in itertools.combinations_with_replacement([1,0], len(vis_nodes))]
    expected_count = dict.fromkeys(hidden_nodes,{})
    
    for _ in xrange(max_iter):
        
        # calculate expected counts
        for perm in data_perm: 
            conditional_prob = model.predict_proba(dict(zip(vis_nodes,perm)))
            for i, node in enumerate(hidden_nodes):
                expected_count[node][perm] = conditional_prob[ind_hidden_nodes[i]].probability(1)
        
        # summing over expected counts
        
        
        # update the parameters  
        
    


In [17]:
x = np.asarray([[1,np.nan],[2,np.nan]])

In [14]:
np.isnan(x)

array([[False,  True],
       [False,  True]], dtype=bool)

In [21]:
col_list = np.all(np.isnan(x),axis=0)

In [40]:
col_list

array([False,  True], dtype=bool)

In [36]:
ind = np.where(col_list)

In [44]:
ind[0][0]

1

In [29]:
np.size(ind)

1

In [48]:
dict.fromkeys(['a','b'],{})

{'a': {}, 'b': {}}

In [52]:
test_model.node_count()

2

In [51]:
np.where(np.asarray([1,0,1,0]))

AttributeError: 'tuple' object has no attribute 'shape'

In [57]:
np.arange(10)[([0,3])]

array([0, 3])

In [38]:
zip([1,0],[1,0])

[(1, 1), (0, 0)]

In [43]:
[1,0]*2

[1, 0, 1, 0]

In [44]:
[x for x in itertools.product([1,0],[1,0],[1,0])]

[(1, 1, 1),
 (1, 1, 0),
 (1, 0, 1),
 (1, 0, 0),
 (0, 1, 1),
 (0, 1, 0),
 (0, 0, 1),
 (0, 0, 0)]

In [50]:
[x for x in itertools.combinations_with_replacement([1,0],3)]

[(1, 1, 1), (1, 1, 0), (1, 0, 0), (0, 0, 0)]

In [41]:
data_perm = [x for x in itertools.combinations_with_replacement([1,0], 2)]

In [42]:
data_perm

[(1, 1), (1, 0), (0, 0)]

In [46]:
dict(zip(name_list,data_perm[0]))

{'A': 1, 'B': 1}

In [68]:
data = np.asarray([[1,np.nan],[0,np.nan]])

In [73]:
em(test_model, data, max_iter=1)

Hidden nodes :  ['B']
{'B': {(0,): 0.59999999999999998, (1,): 0.69999999999999984}}


In [30]:
np.asarray([s.name for s in test_model.states])

array(['A', 'B'], 
      dtype='|S1')

In [16]:
np.all(np.isnan())

True

In [20]:
np.isnan(np.asarray([[np.nan,1],[np.nan,0]]))

array([[ True, False],
       [ True, False]], dtype=bool)

In [23]:
np.all(np.isnan(np.asarray([[np.nan,1],[np.nan,0]])),axis=0)

array([ True, False], dtype=bool)